In [42]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import psycopg2
import datetime

In [43]:
dbname = "kddi_18_param_custom2"
npyname = "trkn_18"
datedir = "230120"
sttime = "2022-04-01 18:00:00"

In [44]:
#解析結果の読み込み
timeseries_sep = np.load("./" + datedir+'/'+ npyname + ".npy")

In [45]:
timeseries_sep.shape

(24, 207)

In [46]:
timeseries_sep

array([[ 88,  62,  95, ..., 109, 109,   1],
       [ 77,  74,  90, ..., 116, 116,   0],
       [ 64,  74,  86, ..., 106, 106,   0],
       ...,
       [ 13,   4,  24, ...,  26,  26,   0],
       [ 21,  12,  12, ...,  28,  28,   0],
       [ 16,   6,  11, ...,  23,  23,   0]], dtype=int64)

In [47]:
#メッシュ情報の読み込み
csv_file = r'./'+datedir+'/'+ npyname+'_header.csv'
df = pd.read_csv(csv_file, dtype={'date':str}, index_col=0, parse_dates=True)
df_i = df.loc["gridcode"] #gridcode #end_mesh_area

In [48]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [ ]:
dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')

for lp in range(4):
    cur =connection.cursor()
    curs =connection.cursor()

    checkMeshtbranList = [] #確認対象メッシュのKDDI値

    #まずは時刻ごとのメッシュ一覧を取得
    sql = "select mesh from " + dbname + " WHERE times = '"+str(dte)+"'"
    cur.execute(sql)
    meshLists = cur.fetchall()
    
    print("time =" + str(dte) + " target =" + str(len(meshLists)))
    for mesh in meshLists:
        
        meshkey = str(int(mesh[0]))
        
        #トラカン値を持つメッシュならば解析しない
        find = 0
        for chkm in df_i.values:
            if str(int(chkm)) == meshkey:
                find = 1
                break
        if find == 1:
            continue
            
        if meshkey <= "50325626324":
            continue

        sql = "select kddi_tran.tbran, T.gridcode,T.断面交通量,kddi_tran.mesh "
        sql += "from " + dbname + " AS kddi_tran "
        sql += "INNER JOIN pointdata as a ON kddi_tran.mesh = cast(a.grid_code AS BIGINT) "
        sql += "CROSS JOIN LATERAL "
        sql += "("
        sql += "SELECT 時刻, gridcode , 断面交通量 from trakan2204_z3out AS trakan_tran "
        sql += "inner JOIN pointdata as b ON trakan_tran.gridcode = cast(grid_code AS BIGINT) "
        sql += "WHERE 時刻 = '"+str(dte)+"' "
        sql += "ORDER BY a.geom <-> b.geom, ST_Distance(a.geom, b.geom) "
        sql += "LIMIT 1) AS T "
        sql += "WHERE times = '"+str(dte)+"' and kddi_tran.mesh = " + meshkey
        curs.execute(sql)
        datalists = curs.fetchall()
        #print(datalists) #(88.0, 50324526133, 13) #対象MESH値、確認対象メッシュ、対象の断面交通量
        
        if len(datalists) == 0:
            #print(sql)
            continue

        print(meshkey)
        #確認対象メッシュの kddi_tran.tbran値
        enabledcheck = meshkey in checkMeshtbranList
        baseMeshItem = 0
        if enabledcheck == True:
            for meshitem in checkMeshtbranList:
                if str(int(mesh)) == meshitem[0]:
                    baseMeshItem = meshitem[1]
                    break
        else:
            cur3 =connection.cursor()
            sql = "select kddi_tran.tbran from " + dbname + " AS kddi_tran "
            sql += "WHERE times = '"+str(dte)+"' and kddi_tran.mesh = " + str(datalists[0][1])
            cur3.execute(sql)
            baseMeshParam = cur3.fetchall()
            setparam = 0
            if len(baseMeshParam) != 0:
                setparam = baseMeshParam[0][0]
            if len(checkMeshtbranList) == 0:
                checkMeshtbranList = [meshkey , setparam]
            else:
                checkMeshtbranList.append([meshkey , setparam])
            cur3.close()
            baseMeshItem = setparam

        #メッシュ値を算出
        if baseMeshItem == 0:
            param = 0
        else:
            param = datalists[0][0] / baseMeshItem * datalists[0][2]
        print(str(datalists[0][0]) + ":" +  str(baseMeshItem) + ":" +  str( datalists[0][2]) )
            
        sql = "UPDATE " + dbname + " set tbran = "+ str(param) + " where times = '"  + str(dte) + "' and mesh = " + meshkey + ";"
        #print(sql)
        cur.execute(sql)
        connection.commit()
    cur.close()
    curs.close()
    dte = dte + datetime.timedelta(minutes=15)
connection.commit()

In [ ]:
#ConvertMeshtoRoad

In [ ]:
connection.close()